In [1]:
import tkinter as tk
from tkinter import ttk, scrolledtext, messagebox, filedialog
from datetime import datetime, timedelta
import json
import os
from pathlib import Path

# For PDF export
try:
    from reportlab.lib.pagesizes import letter, A4
    from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
    from reportlab.lib.units import inch
    from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, PageBreak
    from reportlab.lib.enums import TA_LEFT, TA_CENTER, TA_RIGHT
    PDF_AVAILABLE = True
except ImportError:
    PDF_AVAILABLE = False

# For Word export
try:
    from docx import Document
    from docx.shared import Pt, Inches
    from docx.enum.text import WD_ALIGN_PARAGRAPH
    WORD_AVAILABLE = True
except ImportError:
    WORD_AVAILABLE = False

# For email
try:
    import smtplib
    from email.mime.text import MIMEText
    from email.mime.multipart import MIMEMultipart
    from email.mime.base import MIMEBase
    from email import encoders
    EMAIL_AVAILABLE = True
except ImportError:
    EMAIL_AVAILABLE = False


class DocumentAutomationSystem:
    def __init__(self, root):
        self.root = root
        self.root.title("Professional Document Automation System")
        self.root.geometry("1400x800")
        self.root.configure(bg="#f0f4f8")
        
        # Create data directory for saved forms
        self.data_dir = Path("document_data")
        self.data_dir.mkdir(exist_ok=True)
        
        # Document templates organized by category
        self.document_types = {
            "Audit": [
                ("engagement_letter", "Engagement Letter for Audit"),
                ("audit_report", "Audit Report")
            ],
            "Certificates": [
                ("turnover_cert", "Certificate of Turnover"),
                ("tax_deduction_cert", "Tax Deduction Certificate")
            ],
            "Tax": [
                ("notice_reply", "Notice Reply (Section 143(1))"),
                ("tax_computation", "Tax Computation Statement")
            ],
            "Billing": [
                ("professional_invoice", "Professional Invoice")
            ],
            "Reporting": [
                ("financial_summary", "Summary of Financial Statements")
            ],
            "Legal": [
                ("lease_agreement", "Lease Agreement"),
                ("partnership_deed", "Partnership Deed")
            ]
        }
        
        # Form fields for each document type
        self.form_fields = {
            "engagement_letter": [
                ("Client Name", "text", True),
                ("Client Address", "textarea", True),
                ("Financial Year", "text", True),
                ("Your Firm Name", "text", True),
                ("Firm Address", "textarea", True),
                ("Partner Name", "text", True),
                ("Date", "date", True)
            ],
            "turnover_cert": [
                ("Client Name", "text", True),
                ("PAN", "text", True),
                ("Financial Year", "text", True),
                ("Total Turnover (Rs)", "number", True),
                ("CA Firm Name", "text", True),
                ("Membership No.", "text", True),
                ("Place", "text", True),
                ("Date", "date", True)
            ],
            "notice_reply": [
                ("Assessing Officer Name/Designation", "text", True),
                ("AO Office Address", "textarea", True),
                ("Notice Number", "text", True),
                ("Notice Date", "date", True),
                ("Assessee Name", "text", True),
                ("PAN", "text", True),
                ("Assessment Year", "text", True),
                ("Reply Content/Points", "textarea", True)
            ],
            "professional_invoice": [
                ("Invoice Number", "text", True),
                ("Invoice Date", "date", True),
                ("Client Name", "text", True),
                ("Client Address", "textarea", True),
                ("Services Provided (line by line: Service - Amount)", "textarea", True),
                ("Your Firm Name", "text", True),
                ("GSTIN (if applicable)", "text", False),
                ("Bank Account Details", "textarea", True)
            ],
            "financial_summary": [
                ("Company Name", "text", True),
                ("Financial Year", "text", True),
                ("Total Revenue (Rs)", "number", True),
                ("Total Expenses (Rs)", "number", True),
                ("Net Profit (Rs)", "number", True),
                ("Total Assets (Rs)", "number", True),
                ("Total Liabilities (Rs)", "number", True)
            ],
            "lease_agreement": [
                ("Lessor Name", "text", True),
                ("Lessee Name", "text", True),
                ("Property Address", "textarea", True),
                ("Monthly Rent (Rs)", "number", True),
                ("Security Deposit (Rs)", "number", True),
                ("Lease Period (months)", "number", True),
                ("Lease Start Date", "date", True),
                ("Place", "text", True),
                ("Agreement Date", "date", True)
            ],
            "audit_report": [
                ("Company Name", "text", True),
                ("Company Address", "textarea", True),
                ("Financial Year", "text", True),
                ("Report Date", "date", True),
                ("CA Firm Name", "text", True),
                ("Firm Registration No.", "text", True),
                ("Partner Name", "text", True),
                ("Membership No.", "text", True),
                ("Place", "text", True),
                ("Opinion Type", "text", True)
            ],
            "tax_deduction_cert": [
                ("Deductor Name", "text", True),
                ("Deductor TAN", "text", True),
                ("Deductee Name", "text", True),
                ("Deductee PAN", "text", True),
                ("Financial Year", "text", True),
                ("Quarter", "text", True),
                ("Amount Paid (Rs)", "number", True),
                ("TDS Deducted (Rs)", "number", True),
                ("Section", "text", True),
                ("Certificate Number", "text", True),
                ("Date of Deduction", "date", True),
                ("Date of Payment", "date", True)
            ],
            "partnership_deed": [
                ("Partnership Firm Name", "text", True),
                ("Principal Place of Business", "textarea", True),
                ("Nature of Business", "text", True),
                ("Commencement Date", "date", True),
                ("Partner 1 Name", "text", True),
                ("Partner 1 Address", "textarea", True),
                ("Partner 2 Name", "text", True),
                ("Partner 2 Address", "textarea", True),
                ("Capital Contribution Partner 1 (Rs)", "number", True),
                ("Capital Contribution Partner 2 (Rs)", "number", True),
                ("Profit Sharing Ratio", "text", True),
                ("Place", "text", True),
                ("Date", "date", True)
            ]
        }
        
        self.current_doc_type = None
        self.form_entries = {}
        
        self.setup_ui()
        self.check_dependencies()
    
    def check_dependencies(self):
        """Check and warn about missing dependencies"""
        missing = []
        if not PDF_AVAILABLE:
            missing.append("reportlab (for PDF export)")
        if not WORD_AVAILABLE:
            missing.append("python-docx (for Word export)")
        
        if missing:
            msg = "Optional features disabled. Install:\n\n"
            msg += "\n".join([f"pip install {lib.split('(')[0].strip()}" for lib in missing])
            print(msg)
    
    def setup_ui(self):
        # Header
        header_frame = tk.Frame(self.root, bg="#4F46E5", height=80)
        header_frame.pack(fill="x", padx=0, pady=0)
        header_frame.pack_propagate(False)
        
        title_label = tk.Label(
            header_frame,
            text="Professional Document Automation System",
            font=("Arial", 20, "bold"),
            bg="#4F46E5",
            fg="white"
        )
        title_label.pack(pady=10)
        
        subtitle_label = tk.Label(
            header_frame,
            text="Generate CA, Tax, Audit & Legal Documents Instantly",
            font=("Arial", 11),
            bg="#4F46E5",
            fg="#E0E7FF"
        )
        subtitle_label.pack()
        
        # Main container
        main_container = tk.Frame(self.root, bg="#f0f4f8")
        main_container.pack(fill="both", expand=True, padx=10, pady=10)
        
        # Left panel with tabs
        left_panel = tk.Frame(main_container, bg="white", relief="solid", borderwidth=1)
        left_panel.pack(side="left", fill="both", padx=(0, 5), pady=0, expand=False)
        left_panel.configure(width=400)
        
        left_header = tk.Label(
            left_panel,
            text="Select Document Type",
            font=("Arial", 14, "bold"),
            bg="white",
            fg="#1F2937"
        )
        left_header.pack(pady=10, padx=10, anchor="w")
        
        # Create notebook (tabs)
        style = ttk.Style()
        style.theme_create("custom", parent="alt", settings={
            "TNotebook": {
                "configure": {"tabmargins": [2, 5, 2, 0], "background": "white"}
            },
            "TNotebook.Tab": {
                "configure": {
                    "padding": [10, 5],
                    "font": ("Arial", 10, "bold"),
                    "background": "#E5E7EB"
                },
                "map": {
                    "background": [("selected", "#4F46E5")],
                    "foreground": [("selected", "white"), ("!selected", "#374151")]
                }
            }
        })
        style.theme_use("custom")
        
        self.notebook = ttk.Notebook(left_panel)
        self.notebook.pack(fill="both", expand=True, padx=5, pady=5)
        
        # Create a tab for each category
        for category, docs in self.document_types.items():
            # Create frame for this tab
            tab_frame = tk.Frame(self.notebook, bg="white")
            self.notebook.add(tab_frame, text=category)
            
            # Create scrollable canvas for documents
            canvas = tk.Canvas(tab_frame, bg="white", highlightthickness=0)
            scrollbar = ttk.Scrollbar(tab_frame, orient="vertical", command=canvas.yview)
            scrollable_frame = tk.Frame(canvas, bg="white")
            
            scrollable_frame.bind(
                "<Configure>",
                lambda e, c=canvas: c.configure(scrollregion=c.bbox("all"))
            )
            
            canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
            canvas.configure(yscrollcommand=scrollbar.set)
            
            # Add document buttons
            for doc_id, doc_name in docs:
                btn = tk.Button(
                    scrollable_frame,
                    text=doc_name,
                    font=("Arial", 11),
                    bg="#F9FAFB",
                    fg="#1F2937",
                    relief="flat",
                    anchor="w",
                    padx=20,
                    pady=15,
                    command=lambda d=doc_id: self.select_document(d),
                    cursor="hand2",
                    activebackground="#E0E7FF"
                )
                btn.pack(fill="x", padx=10, pady=5)
                
                # Hover effect
                btn.bind("<Enter>", lambda e, b=btn: b.configure(bg="#E5E7EB"))
                btn.bind("<Leave>", lambda e, b=btn: b.configure(bg="#F9FAFB"))
            
            canvas.pack(side="left", fill="both", expand=True)
            scrollbar.pack(side="right", fill="y")
        
        # Right panel - Form and output
        right_panel = tk.Frame(main_container, bg="#f0f4f8")
        right_panel.pack(side="left", fill="both", expand=True, padx=(5, 0))
        
        # Form panel
        self.form_panel = tk.Frame(right_panel, bg="white", relief="solid", borderwidth=1)
        self.form_panel.pack(fill="both", expand=True, pady=(0, 5))
        
        form_header_frame = tk.Frame(self.form_panel, bg="white")
        form_header_frame.pack(fill="x", pady=10, padx=10)
        
        form_header = tk.Label(
            form_header_frame,
            text="Fill Document Details",
            font=("Arial", 14, "bold"),
            bg="white",
            fg="#1F2937"
        )
        form_header.pack(side="left")
        
        # Save/Load buttons
        btn_frame = tk.Frame(form_header_frame, bg="white")
        btn_frame.pack(side="right")
        
        save_btn = tk.Button(
            btn_frame,
            text="💾 Save Form",
            font=("Arial", 9),
            bg="#3B82F6",
            fg="white",
            relief="flat",
            padx=10,
            pady=5,
            command=self.save_form_data,
            cursor="hand2"
        )
        save_btn.pack(side="left", padx=2)
        
        load_btn = tk.Button(
            btn_frame,
            text="📂 Load Form",
            font=("Arial", 9),
            bg="#8B5CF6",
            fg="white",
            relief="flat",
            padx=10,
            pady=5,
            command=self.load_form_data,
            cursor="hand2"
        )
        load_btn.pack(side="left", padx=2)
        
        # Scrollable form area
        self.form_canvas = tk.Canvas(self.form_panel, bg="white", highlightthickness=0)
        form_scrollbar = ttk.Scrollbar(self.form_panel, orient="vertical", command=self.form_canvas.yview)
        self.form_frame = tk.Frame(self.form_canvas, bg="white")
        
        self.form_frame.bind(
            "<Configure>",
            lambda e: self.form_canvas.configure(scrollregion=self.form_canvas.bbox("all"))
        )
        
        self.form_canvas.create_window((0, 0), window=self.form_frame, anchor="nw")
        self.form_canvas.configure(yscrollcommand=form_scrollbar.set)
        
        self.form_canvas.pack(side="left", fill="both", expand=True, padx=10)
        form_scrollbar.pack(side="right", fill="y")
        
        # Generate button
        self.generate_btn = tk.Button(
            self.form_panel,
            text="✨ Generate Document",
            font=("Arial", 12, "bold"),
            bg="#4F46E5",
            fg="white",
            relief="flat",
            padx=20,
            pady=12,
            command=self.generate_document,
            cursor="hand2"
        )
        self.generate_btn.pack(pady=10)
        self.generate_btn.pack_forget()
        
        # Output panel
        output_panel = tk.Frame(right_panel, bg="white", relief="solid", borderwidth=1)
        output_panel.pack(fill="both", expand=True)
        
        output_header_frame = tk.Frame(output_panel, bg="white")
        output_header_frame.pack(fill="x", pady=10, padx=10)
        
        output_header = tk.Label(
            output_header_frame,
            text="Generated Document",
            font=("Arial", 14, "bold"),
            bg="white",
            fg="#1F2937"
        )
        output_header.pack(side="left")
        
        # Action buttons
        action_frame = tk.Frame(output_header_frame, bg="white")
        action_frame.pack(side="right")
        
        copy_btn = tk.Button(
            action_frame,
            text="📋 Copy",
            font=("Arial", 9),
            bg="#10B981",
            fg="white",
            relief="flat",
            padx=10,
            pady=5,
            command=self.copy_to_clipboard,
            cursor="hand2"
        )
        copy_btn.pack(side="left", padx=2)
        
        if WORD_AVAILABLE:
            word_btn = tk.Button(
                action_frame,
                text="📄 Export Word",
                font=("Arial", 9),
                bg="#2563EB",
                fg="white",
                relief="flat",
                padx=10,
                pady=5,
                command=self.export_to_word,
                cursor="hand2"
            )
            word_btn.pack(side="left", padx=2)
        
        if PDF_AVAILABLE:
            pdf_btn = tk.Button(
                action_frame,
                text="📕 Export PDF",
                font=("Arial", 9),
                bg="#DC2626",
                fg="white",
                relief="flat",
                padx=10,
                pady=5,
                command=self.export_to_pdf,
                cursor="hand2"
            )
            pdf_btn.pack(side="left", padx=2)
        
        if EMAIL_AVAILABLE:
            email_btn = tk.Button(
                action_frame,
                text="📧 Email",
                font=("Arial", 9),
                bg="#F59E0B",
                fg="white",
                relief="flat",
                padx=10,
                pady=5,
                command=self.email_document,
                cursor="hand2"
            )
            email_btn.pack(side="left", padx=2)
        
        # Output text area
        self.output_text = scrolledtext.ScrolledText(
            output_panel,
            font=("Courier New", 10),
            bg="#F9FAFB",
            fg="#1F2937",
            wrap="word",
            padx=10,
            pady=10
        )
        self.output_text.pack(fill="both", expand=True, padx=10, pady=(0, 10))
        self.output_text.insert("1.0", "Select a document type from the tabs on the left and fill the form to generate your document.")
        self.output_text.config(state="disabled")
    
    def select_document(self, doc_type):
        self.current_doc_type = doc_type
        self.form_entries = {}
        
        # Clear form frame
        for widget in self.form_frame.winfo_children():
            widget.destroy()
        
        # Get fields for this document type
        if doc_type not in self.form_fields:
            return
        
        fields = self.form_fields[doc_type]
        
        # Create form fields
        for i, (label, field_type, required) in enumerate(fields):
            # Label
            label_text = label + (" *" if required else "")
            lbl = tk.Label(
                self.form_frame,
                text=label_text,
                font=("Arial", 10, "bold"),
                bg="white",
                fg="#374151",
                anchor="w"
            )
            lbl.grid(row=i*2, column=0, sticky="w", pady=(10, 5), padx=5)
            
            # Input field
            if field_type == "textarea":
                entry = tk.Text(
                    self.form_frame,
                    font=("Arial", 10),
                    height=3,
                    width=50,
                    relief="solid",
                    borderwidth=1
                )
            else:
                entry = tk.Entry(
                    self.form_frame,
                    font=("Arial", 10),
                    width=50,
                    relief="solid",
                    borderwidth=1
                )
            
            entry.grid(row=i*2+1, column=0, sticky="ew", pady=(0, 5), padx=5)
            self.form_entries[label] = (entry, field_type)
        
        # Show generate button
        self.generate_btn.pack(pady=10)
    
    def save_form_data(self):
        """Save current form data to JSON file"""
        if not self.current_doc_type:
            messagebox.showwarning("Warning", "Please select a document type first.")
            return
        
        # Collect form data
        form_data = {
            "document_type": self.current_doc_type,
            "fields": {}
        }
        
        for label, (entry, field_type) in self.form_entries.items():
            if field_type == "textarea":
                value = entry.get("1.0", "end-1c").strip()
            else:
                value = entry.get().strip()
            form_data["fields"][label] = value
        
        # Ask for filename
        filename = filedialog.asksaveasfilename(
            initialdir=self.data_dir,
            title="Save Form Data",
            defaultextension=".json",
            filetypes=[("JSON files", "*.json"), ("All files", "*.*")]
        )
        
        if filename:
            try:
                with open(filename, 'w') as f:
                    json.dump(form_data, f, indent=2)
                messagebox.showinfo("Success", f"Form data saved successfully!")
            except Exception as e:
                messagebox.showerror("Error", f"Failed to save form data:\n{str(e)}")
    
    def load_form_data(self):
        """Load form data from JSON file"""
        filename = filedialog.askopenfilename(
            initialdir=self.data_dir,
            title="Load Form Data",
            filetypes=[("JSON files", "*.json"), ("All files", "*.*")]
        )
        
        if filename:
            try:
                with open(filename, 'r') as f:
                    form_data = json.load(f)
                
                # Select the document type
                doc_type = form_data.get("document_type")
                if doc_type:
                    self.select_document(doc_type)
                    
                    # Fill in the form fields
                    for label, value in form_data.get("fields", {}).items():
                        if label in self.form_entries:
                            entry, field_type = self.form_entries[label]
                            if field_type == "textarea":
                                entry.delete("1.0", "end")
                                entry.insert("1.0", value)
                            else:
                                entry.delete(0, "end")
                                entry.insert(0, value)
                    
                    messagebox.showinfo("Success", "Form data loaded successfully!")
                else:
                    messagebox.showwarning("Warning", "Invalid form data file.")
            except Exception as e:
                messagebox.showerror("Error", f"Failed to load form data:\n{str(e)}")
    
    def generate_document(self):
        if not self.current_doc_type:
            return
        
        # Collect form data
        form_data = {}
        for label, (entry, field_type) in self.form_entries.items():
            if field_type == "textarea":
                value = entry.get("1.0", "end-1c").strip()
            else:
                value = entry.get().strip()
            form_data[label] = value
        
        # Generate document based on type
        doc_text = self.generate_document_text(self.current_doc_type, form_data)
        
        # Update output
        self.output_text.config(state="normal")
        self.output_text.delete("1.0", "end")
        self.output_text.insert("1.0", doc_text)
        self.output_text.config(state="disabled")
    
    def generate_document_text(self, doc_type, data):
        if doc_type == "engagement_letter":
            return f"""ENGAGEMENT LETTER FOR STATUTORY AUDIT

Date: {data.get('Date', '__________')}

To,
{data.get('Client Name', '<Client Name>')}
{data.get('Client Address', '<Client Address>')}

Dear Sir/Madam,

Subject: Engagement Letter for Statutory Audit for the Financial Year {data.get('Financial Year', '<FY>')}

We are pleased to confirm our acceptance and understanding of the terms of our engagement as statutory auditors of {data.get('Client Name', '<Client Name>')} for the financial year {data.get('Financial Year', '<FY>')}.

1. SCOPE OF AUDIT

1.1 We will conduct our audit in accordance with the Standards on Auditing issued by the Institute of Chartered Accountants of India (ICAI).

1.2 Our audit will be conducted to express an opinion on the financial statements prepared by the management.

1.3 The audit will include examination, on a test basis, of evidence supporting the amounts and disclosures in the financial statements.

2. MANAGEMENT RESPONSIBILITIES

2.1 The management is responsible for the preparation and fair presentation of financial statements.

2.2 The management is responsible for maintaining adequate accounting records and internal control systems.

2.3 The management will provide us with access to all information, records, and documents required for the audit.

3. AUDITOR RESPONSIBILITIES

3.1 We will plan and perform the audit to obtain reasonable assurance about whether the financial statements are free from material misstatement.

3.2 We will communicate audit findings and required disclosures in accordance with professional standards.

3.3 We will maintain confidentiality of all information obtained during the audit.

4. LIMITATIONS OF AUDIT

4.1 An audit includes examining evidence on a test basis and cannot be expected to detect all errors or frauds.

4.2 Management retains responsibility for prevention and detection of fraud.

5. FEES AND BILLING

5.1 Our professional fees will be billed as agreed separately.

5.2 Out-of-pocket expenses will be billed additionally as incurred.

6. REPORT

6.1 Our audit report will be issued in accordance with the requirements of the Companies Act, 2013 and Standards on Auditing.

We look forward to working with you and your team. Please confirm your agreement with the terms of this engagement by signing and returning the attached copy of this letter.

Yours faithfully,

For {data.get('Your Firm Name', '<Firm Name>')}
Chartered Accountants

{data.get('Partner Name', '<Partner Name>')}
Partner

{data.get('Firm Address', '<Firm Address>')}


ACCEPTANCE BY CLIENT

We have read and understood the terms of engagement as stated above and agree to the same.

For {data.get('Client Name', '<Client Name>')}

_______________________
Authorized Signatory
Name:
Designation:
Date:
"""
        
        elif doc_type == "turnover_cert":
            turnover = data.get('Total Turnover (Rs)', '0')
            return f"""CERTIFICATE OF TURNOVER

To Whomsoever It May Concern

This is to certify that we have examined the books of accounts and other relevant records of:

Name: {data.get('Client Name', '<Client Name>')}
PAN: {data.get('PAN', '<PAN>')}
Financial Year: {data.get('Financial Year', '<FY>')}

Based on our examination and according to the information and explanations given to us, we certify that:

1. The Total Turnover/Gross Receipts of {data.get('Client Name', '<Client Name>')} for the Financial Year {data.get('Financial Year', '<FY>')} is:

   Rs. {self.format_number(turnover)} (Rupees {self.number_to_words(turnover)} only)

2. This certificate is issued based on the books of accounts and records maintained by the assessee and verified by us.

3. This certificate is being issued for official purposes as requested by the above-named entity.


For {data.get('CA Firm Name', '<CA Firm Name>')}
Chartered Accountants
Firm Registration No.: __________

{data.get('Membership No.', '<Membership No.>')}
Partner
Membership No.: {data.get('Membership No.', '<Membership No.')}
UDIN: __________

Place: {data.get('Place', '<Place>')}
Date: {data.get('Date', '__________')}
"""
        
        elif doc_type == "notice_reply":
            return f"""{data.get('Assessing Officer Name/Designation', '<AO Name>')}
{data.get('AO Office Address', '<AO Address>')}

Date: {datetime.now().strftime('%d/%m/%Y')}

Subject: Reply to Notice under Section 143(1) of the Income Tax Act, 1961

Reference: Notice No. {data.get('Notice Number', '<Notice No>')} dated {data.get('Notice Date', '<Date>')}

Dear Sir/Madam,

With reference to the above notice issued under Section 143(1) of the Income Tax Act, 1961, we submit our reply as follows:

PARTICULARS OF THE ASSESSEE:

Name: {data.get('Assessee Name', '<Assessee Name>')}
PAN: {data.get('PAN', '<PAN>')}
Assessment Year: {data.get('Assessment Year', '<AY>')}

REPLY TO THE NOTICE:

{data.get('Reply Content/Points', '<Reply content here>')}

DOCUMENTS ENCLOSED:

1. Copy of the original notice
2. Supporting documents as applicable
3. Revised computation (if applicable)

We trust the above clarification and documents submitted are in order. We request you to kindly consider the same and process the assessment accordingly.

In case you require any further information or clarification, please feel free to contact the undersigned.

Thanking you,

Yours faithfully,


_______________________
Authorized Signatory
Name: {data.get('Assessee Name', '<Assessee Name>')}
PAN: {data.get('PAN', '<PAN>')}

Contact Details:
Mobile: __________
Email: __________
"""
        
        elif doc_type == "professional_invoice":
            services = data.get('Services Provided (line by line: Service - Amount)', '').split('\n')
            total = 0
            service_lines = []
            
            for i, service in enumerate(services, 1):
                if '-' in service:
                    parts = service.split('-')
                    desc = parts[0].strip()
                    try:
                        amount = float(parts[1].strip())
                        total += amount
                        service_lines.append(f"{i}.     {desc.ljust(45)} {self.format_number(str(amount)).rjust(10)}")
                    except:
                        service_lines.append(f"{i}.     {service.ljust(45)} {'0.00'.rjust(10)}")
            
            gst = total * 0.18
            grand_total = total + gst
            
            return f"""TAX INVOICE

Invoice No: {data.get('Invoice Number', '<Invoice No>')}
Date: {data.get('Invoice Date', '__________')}

BILLED TO:
{data.get('Client Name', '<Client Name>')}
{data.get('Client Address', '<Client Address>')}

BILLED BY:
{data.get('Your Firm Name', '<Firm Name>')}
{('GSTIN: ' + data.get('GSTIN (if applicable)', '')) if data.get('GSTIN (if applicable)') else ''}

────────────────────────────────────────────────────────────────

PARTICULARS OF SERVICES

S.No.  Description                                    Amount (Rs)
────────────────────────────────────────────────────────────────
{chr(10).join(service_lines)}
────────────────────────────────────────────────────────────────

                                          Subtotal: {self.format_number(str(total)).rjust(10)}
                                    GST @ 18%: {self.format_number(str(gst)).rjust(10)}
────────────────────────────────────────────────────────────────
                                      GRAND TOTAL: {self.format_number(str(grand_total)).rjust(10)}
────────────────────────────────────────────────────────────────

Amount in Words: {self.number_to_words(str(grand_total))} only

BANK DETAILS:
{data.get('Bank Account Details', '<Bank Details>')}

TERMS & CONDITIONS:
1. Payment is due within 15 days of invoice date.
2. Please make payment in favor of {data.get('Your Firm Name', '<Firm Name>')}.
3. All disputes subject to local jurisdiction.

For {data.get('Your Firm Name', '<Firm Name>')}


_______________________
Authorized Signatory
"""
        
        elif doc_type == "financial_summary":
            revenue = float(data.get('Total Revenue (Rs)', 0) or 0)
            expenses = float(data.get('Total Expenses (Rs)', 0) or 0)
            net_profit = float(data.get('Net Profit (Rs)', 0) or 0)
            assets = float(data.get('Total Assets (Rs)', 0) or 0)
            liabilities = float(data.get('Total Liabilities (Rs)', 0) or 0)
            equity = assets - liabilities
            
            npm = (net_profit / revenue * 100) if revenue else 0
            roa = (net_profit / assets * 100) if assets else 0
            debt_equity = (liabilities / equity) if equity else 0
            
            return f"""SUMMARY OF FINANCIAL STATEMENTS

Company Name: {data.get('Company Name', '<Company Name>')}
Financial Year: {data.get('Financial Year', '<FY>')}

════════════════════════════════════════════════════════════════

PROFIT & LOSS SUMMARY

Revenue
Total Revenue                                  Rs. {self.format_number(str(revenue))}

Expenses
Total Expenses                                 Rs. {self.format_number(str(expenses))}
                                              ─────────────────
Net Profit                                     Rs. {self.format_number(str(net_profit))}
                                              ═════════════════

BALANCE SHEET SUMMARY

Assets
Total Assets                                   Rs. {self.format_number(str(assets))}

Liabilities & Equity
Total Liabilities                              Rs. {self.format_number(str(liabilities))}
Shareholders' Equity                           Rs. {self.format_number(str(equity))}
                                              ─────────────────
Total Liabilities & Equity                     Rs. {self.format_number(str(assets))}
                                              ═════════════════

KEY FINANCIAL RATIOS

Net Profit Margin                              {npm:.2f}%
Return on Assets (ROA)                         {roa:.2f}%
Debt to Equity Ratio                           {debt_equity:.2f}

════════════════════════════════════════════════════════════════

Note: This is a summary document. Detailed financial statements available separately.
"""
        
        elif doc_type == "lease_agreement":
            try:
                start_date = datetime.strptime(data.get('Lease Start Date', '2024-01-01'), '%Y-%m-%d') if data.get('Lease Start Date') else datetime.now()
            except:
                start_date = datetime.now()
            
            lease_months = int(data.get('Lease Period (months)', 12) or 12)
            end_date = start_date + timedelta(days=lease_months*30)
            
            return f"""LEASE AGREEMENT

This Lease Agreement is made on {data.get('Agreement Date', '__________')} at {data.get('Place', '<Place>')}

BETWEEN

{data.get('Lessor Name', '<Lessor Name>')} (hereinafter referred to as "LESSOR" or "LANDLORD") of the First Part

AND

{data.get('Lessee Name', '<Lessee Name>')} (hereinafter referred to as "LESSEE" or "TENANT") of the Second Part

WHEREAS the Lessor is the lawful owner of the property described herein and is willing to let out the said property to the Lessee on the terms and conditions mentioned below:

1. PROPERTY DESCRIPTION

The property situated at:
{data.get('Property Address', '<Property Address>')}

(hereinafter referred to as "the Premises")

2. LEASE PERIOD

2.1 The lease shall commence from {start_date.strftime('%d/%m/%Y')} and continue for a period of {lease_months} months, ending on {end_date.strftime('%d/%m/%Y')}.

2.2 The lease may be renewed by mutual consent on terms to be agreed upon.

3. RENT AND PAYMENT TERMS

3.1 Monthly Rent: Rs. {self.format_number(data.get('Monthly Rent (Rs)', '0'))} (Rupees {self.number_to_words(data.get('Monthly Rent (Rs)', '0'))} only)

3.2 The rent shall be paid on or before the 5th day of each calendar month.

3.3 Security Deposit: Rs. {self.format_number(data.get('Security Deposit (Rs)', '0'))} (Rupees {self.number_to_words(data.get('Security Deposit (Rs)', '0'))} only)

3.4 The security deposit shall be refunded after adjustment of dues at the end of the lease period.

4. LESSOR'S OBLIGATIONS

4.1 To hand over peaceful possession of the premises to the Lessee.

4.2 To ensure the property is in habitable condition at the commencement of lease.

4.3 To maintain the structural integrity of the building.

5. LESSEE'S OBLIGATIONS

5.1 To pay rent regularly on the due date.

5.2 To use the premises for residential/commercial purposes only.

5.3 To maintain the premises in good condition.

5.4 Not to sublet or assign the premises without written consent of the Lessor.

5.5 To return the premises in the same condition as received, subject to normal wear and tear.

6. MAINTENANCE AND REPAIRS

6.1 The Lessee shall be responsible for minor repairs and routine maintenance.

6.2 Major structural repairs shall be the responsibility of the Lessor.

7. UTILITIES

7.1 Electricity, water, and other utility charges shall be borne by the Lessee.

8. TERMINATION

8.1 Either party may terminate this agreement by giving 2 months' prior written notice.

8.2 In case of breach of terms, the aggrieved party may terminate with 30 days' notice.

9. DISPUTE RESOLUTION

9.1 Any disputes arising out of this agreement shall be resolved through mutual discussion.

9.2 If unresolved, disputes shall be subject to the jurisdiction of courts at {data.get('Place', '<Place>')}.

10. GENERAL PROVISIONS

10.1 This agreement shall be governed by the laws of India.

10.2 Any modifications to this agreement must be made in writing and signed by both parties.

IN WITNESS WHEREOF, both parties have executed this Lease Agreement on the date first written above.


LESSOR                                    LESSEE

_______________________                   _______________________
{data.get('Lessor Name', '<Lessor Name>')}                            {data.get('Lessee Name', '<Lessee Name>')}
Signature                                 Signature


WITNESSES:

1. _______________________               2. _______________________
   Name:                                    Name:
   Address:                                 Address:
   Signature:                               Signature:
"""
        
        elif doc_type == "audit_report":
            return f"""INDEPENDENT AUDITOR'S REPORT

To the Members of
{data.get('Company Name', '<Company Name>')}
{data.get('Company Address', '<Company Address>')}

REPORT ON THE AUDIT OF FINANCIAL STATEMENTS

Opinion

We have audited the accompanying financial statements of {data.get('Company Name', '<Company Name>')}, which comprise the Balance Sheet as at March 31, {data.get('Financial Year', '<FY>')}, the Statement of Profit and Loss (including Other Comprehensive Income), the Statement of Changes in Equity and the Statement of Cash Flows for the year ended on that date, and notes to the financial statements, including a summary of significant accounting policies and other explanatory information.

In our opinion and to the best of our information and according to the explanations given to us, the aforesaid financial statements give the information required by the Companies Act, 2013 in the manner so required and give a true and fair view in conformity with the Indian Accounting Standards prescribed under section 133 of the Act read with the Companies (Indian Accounting Standards) Rules, 2015, as amended, ("Ind AS") and other accounting principles generally accepted in India, of the state of affairs of the Company as at March 31, {data.get('Financial Year', '<FY>')}, and its {data.get('Opinion Type', 'profit/loss')}, total comprehensive income, changes in equity and its cash flows for the year ended on that date.

Basis for Opinion

We conducted our audit of the financial statements in accordance with the Standards on Auditing specified under section 143(10) of the Companies Act, 2013 (the Act). Our responsibilities under those Standards are further described in the Auditor's Responsibilities for the Audit of the Financial Statements section of our report. We are independent of the Company in accordance with the Code of Ethics issued by the Institute of Chartered Accountants of India (ICAI) together with the ethical requirements that are relevant to our audit of the financial statements under the provisions of the Act and the Rules made thereunder, and we have fulfilled our other ethical responsibilities in accordance with these requirements and the ICAI's Code of Ethics.

We believe that the audit evidence we have obtained is sufficient and appropriate to provide a basis for our audit opinion on the financial statements.

Key Audit Matters

Key audit matters are those matters that, in our professional judgment, were of most significance in our audit of the financial statements of the current period. These matters were addressed in the context of our audit of the financial statements as a whole, and in forming our opinion thereon, and we do not provide a separate opinion on these matters.

Information Other than the Financial Statements and Auditor's Report Thereon

The Company's Board of Directors is responsible for the preparation of the other information. The other information comprises the information included in the Management Discussion and Analysis, Board's Report including Annexures to Board's Report, Business Responsibility Report, Corporate Governance and Shareholder's Information, but does not include the financial statements and our auditor's report thereon.

Our opinion on the financial statements does not cover the other information and we do not express any form of assurance conclusion thereon.

Management's Responsibility for the Financial Statements

The Company's Board of Directors is responsible for the matters stated in section 134(5) of the Act with respect to the preparation of these financial statements that give a true and fair view of the financial position, financial performance, total comprehensive income, changes in equity and cash flows of the Company in accordance with the Ind AS and other accounting principles generally accepted in India.

This responsibility also includes maintenance of adequate accounting records in accordance with the provisions of the Act for safeguarding the assets of the Company and for preventing and detecting frauds and other irregularities; selection and application of appropriate accounting policies; making judgments and estimates that are reasonable and prudent; and design, implementation and maintenance of adequate internal financial controls, that were operating effectively for ensuring the accuracy and completeness of the accounting records, relevant to the preparation and presentation of the financial statements that give a true and fair view and are free from material misstatement, whether due to fraud or error.

Auditor's Responsibilities for the Audit of the Financial Statements

Our objectives are to obtain reasonable assurance about whether the financial statements as a whole are free from material misstatement, whether due to fraud or error, and to issue an auditor's report that includes our opinion. Reasonable assurance is a high level of assurance, but is not a guarantee that an audit conducted in accordance with SAs will always detect a material misstatement when it exists.

Report on Other Legal and Regulatory Requirements

As required by Section 143(3) of the Act, based on our audit we report that:

a) We have sought and obtained all the information and explanations which to the best of our knowledge and belief were necessary for the purposes of our audit.

b) In our opinion, proper books of account as required by law have been kept by the Company so far as it appears from our examination of those books.

c) The Balance Sheet, the Statement of Profit and Loss including Other Comprehensive Income, Statement of Changes in Equity and the Statement of Cash Flow dealt with by this Report are in agreement with the relevant books of account.

d) In our opinion, the aforesaid financial statements comply with the Ind AS specified under Section 133 of the Act.

e) On the basis of the written representations received from the directors as on March 31, {data.get('Financial Year', '<FY>')}, and taken on record by the Board of Directors, none of the directors is disqualified as on March 31, {data.get('Financial Year', '<FY>')}, from being appointed as a director in terms of Section 164(2) of the Act.

f) With respect to the adequacy of the internal financial controls over financial reporting of the Company and the operating effectiveness of such controls, refer to our separate Report in "Annexure A".

g) With respect to the other matters to be included in the Auditor's Report in accordance with the requirements of section 197(16) of the Act, as amended, in our opinion and to the best of our information and according to the explanations given to us, the remuneration paid by the Company to its directors during the year is in accordance with the provisions of section 197 of the Act.


For {data.get('CA Firm Name', '<CA Firm Name>')}
Chartered Accountants
Firm Registration Number: {data.get('Firm Registration No.', '<FRN>')}


{data.get('Partner Name', '<Partner Name>')}
Partner
Membership Number: {data.get('Membership No.', '<Membership No.')}
UDIN: __________

Place: {data.get('Place', '<Place>')}
Date: {data.get('Report Date', '__________')}
"""
        
        elif doc_type == "tax_deduction_cert":
            amount_paid = float(data.get('Amount Paid (Rs)', 0) or 0)
            tds = float(data.get('TDS Deducted (Rs)', 0) or 0)
            
            return f"""CERTIFICATE OF TAX DEDUCTED AT SOURCE

[Under Section 203 of the Income-tax Act, 1961]

Certificate No.: {data.get('Certificate Number', '<Certificate No>')}
Financial Year: {data.get('Financial Year', '<FY>')}
Quarter: {data.get('Quarter', '<Quarter>')}

PART A - DETAILS OF DEDUCTOR

Name of Deductor: {data.get('Deductor Name', '<Deductor Name>')}
TAN of Deductor: {data.get('Deductor TAN', '<TAN>')}

PART B - DETAILS OF DEDUCTEE

Name of Deductee: {data.get('Deductee Name', '<Deductee Name>')}
PAN of Deductee: {data.get('Deductee PAN', '<PAN>')}

PART C - DETAILS OF TAX DEDUCTED

Section under which tax deducted: {data.get('Section', '<Section>')}
Date of Deduction: {data.get('Date of Deduction', '__________')}
Date of Payment to Government: {data.get('Date of Payment', '__________')}

Amount Paid/Credited: Rs. {self.format_number(str(amount_paid))}
TDS Deducted: Rs. {self.format_number(str(tds))}
Net Amount Paid: Rs. {self.format_number(str(amount_paid - tds))}

Amount in Words: {self.number_to_words(str(tds))} only

PART D - CERTIFICATE

This is to certify that:

1. Tax has been deducted at source in respect of payment made to the deductee named above.

2. The tax so deducted has been paid to the credit of the Central Government within the prescribed time.

3. The amount of tax deducted as mentioned above has been included in the quarterly TDS return filed by the deductor.

4. The deductee may claim credit of the tax deducted at source while filing the Income Tax Return.

VERIFICATION

I, on behalf of {data.get('Deductor Name', '<Deductor Name>')}, hereby certify that the information given above is true, complete and correct and is based on the books of account, documents and other relevant records.


_______________________
Authorized Signatory
Name:
Designation:
Date:

Note: This certificate has been generated based on the information available in the TDS return filed by the deductor.
"""
        
        elif doc_type == "partnership_deed":
            capital1 = float(data.get('Capital Contribution Partner 1 (Rs)', 0) or 0)
            capital2 = float(data.get('Capital Contribution Partner 2 (Rs)', 0) or 0)
            total_capital = capital1 + capital2
            
            return f"""PARTNERSHIP DEED

This Partnership Deed is executed on {data.get('Date', '__________')} at {data.get('Place', '<Place>')}

BETWEEN

{data.get('Partner 1 Name', '<Partner 1 Name>')}, residing at {data.get('Partner 1 Address', '<Partner 1 Address>')}, hereinafter referred to as "PARTNER 1" (which expression shall unless repugnant to the context or meaning thereof include his/her heirs, executors, administrators and assigns) of the FIRST PART

AND

{data.get('Partner 2 Name', '<Partner 2 Name>')}, residing at {data.get('Partner 2 Address', '<Partner 2 Address>')}, hereinafter referred to as "PARTNER 2" (which expression shall unless repugnant to the context or meaning thereof include his/her heirs, executors, administrators and assigns) of the SECOND PART

(Partner 1 and Partner 2 are hereinafter collectively referred to as "PARTNERS")

WHEREAS the Partners have agreed to carry on business in partnership upon the terms and conditions hereinafter set forth:

NOW THIS DEED WITNESSETH AS FOLLOWS:

1. NAME OF THE FIRM

The Partnership Firm shall be known as:
"{data.get('Partnership Firm Name', '<Partnership Firm Name>')}"

2. NATURE OF BUSINESS

The partnership shall carry on the business of:
{data.get('Nature of Business', '<Nature of Business>')}

3. PLACE OF BUSINESS

The principal place of business of the partnership shall be at:
{data.get('Principal Place of Business', '<Address>')}

The partners may, by mutual consent, establish additional places of business at such other places as may be decided from time to time.

4. COMMENCEMENT AND DURATION

4.1 The partnership shall be deemed to have commenced from {data.get('Commencement Date', '__________')}.

4.2 The partnership shall continue until dissolved by mutual consent or as per the terms of this deed.

5. CAPITAL CONTRIBUTION

5.1 The capital of the partnership shall be Rs. {self.format_number(str(total_capital))} (Rupees {self.number_to_words(str(total_capital))} only).

5.2 The capital contribution of each partner shall be as follows:

    Partner 1: Rs. {self.format_number(str(capital1))} (Rupees {self.number_to_words(str(capital1))} only)
    Partner 2: Rs. {self.format_number(str(capital2))} (Rupees {self.number_to_words(str(capital2))} only)

5.3 The capital contribution shall be made in cash/bank transfer at the time of execution of this deed.

6. PROFIT AND LOSS SHARING

The profits and losses of the partnership business shall be shared by the partners in the ratio of:
{data.get('Profit Sharing Ratio', '<Ratio>')}

7. DRAWINGS

Each partner shall be entitled to draw such amounts from the partnership as may be mutually agreed upon from time to time.

8. RIGHTS AND DUTIES OF PARTNERS

8.1 All partners shall actively participate in the conduct and management of the partnership business.

8.2 All partners shall devote their time, attention and skill to the business of the partnership.

8.3 No partner shall, without the consent of other partners:
    a) Engage in any other business
    b) Enter into any contract on behalf of the firm exceeding Rs. 50,000
    c) Admit any new partner
    d) Transfer or assign their interest in the partnership

9. ACCOUNTS AND AUDIT

9.1 Proper books of account shall be maintained at the principal place of business.

9.2 The accounts of the partnership shall be closed on 31st March every year.

9.3 The accounts shall be audited by a qualified Chartered Accountant to be appointed by mutual consent.

10. BANKING

10.1 All receipts of the partnership shall be deposited in the partnership bank account.

10.2 All payments on behalf of the partnership shall be made from the partnership bank account.

10.3 Cheques/instruments shall require signatures of both/all partners.

11. RETIREMENT AND DEATH

11.1 A partner may retire from the partnership by giving 3 months' written notice to other partners.

11.2 In case of death of a partner, the legal heirs shall be entitled to receive the deceased partner's share after settlement of accounts.

12. DISSOLUTION

The partnership may be dissolved:
a) By mutual consent of all partners
b) By giving 3 months' written notice by any partner
c) On death or insolvency of any partner
d) By order of court

13. ARBITRATION

Any dispute arising out of or in connection with this partnership deed shall be referred to arbitration in accordance with the Arbitration and Conciliation Act, 1996.

14. AMENDMENTS

This deed may be amended only by mutual written consent of all partners.

15. GOVERNING LAW

This partnership deed shall be governed by and construed in accordance with the Indian Partnership Act, 1932 and other applicable laws of India.

IN WITNESS WHEREOF, the parties have executed this Partnership Deed on the date first mentioned above.


PARTNER 1                                 PARTNER 2

_______________________                   _______________________
{data.get('Partner 1 Name', '<Partner 1 Name>')}                            {data.get('Partner 2 Name', '<Partner 2 Name>')}
Signature                                 Signature


WITNESSES:

1. _______________________               2. _______________________
   Name:                                    Name:
   Address:                                 Address:
   Signature:                               Signature:
"""
        
        return "Document type not implemented yet."
    
    def export_to_word(self):
        """Export document to Microsoft Word format"""
        if not WORD_AVAILABLE:
            messagebox.showerror("Error", "python-docx not installed.\nInstall with: pip install python-docx")
            return
        
        doc_text = self.output_text.get("1.0", "end-1c")
        if not doc_text or doc_text.startswith("Select a document"):
            messagebox.showwarning("Warning", "Please generate a document first.")
            return
        
        filename = filedialog.asksaveasfilename(
            title="Save as Word Document",
            defaultextension=".docx",
            filetypes=[("Word Document", "*.docx"), ("All files", "*.*")]
        )
        
        if filename:
            try:
                document = Document()
                
                # Set default font
                style = document.styles['Normal']
                style.font.name = 'Arial'
                style.font.size = Pt(11)
                
                # Add content
                for line in doc_text.split('\n'):
                    if line.strip():
                        paragraph = document.add_paragraph(line)
                    else:
                        document.add_paragraph()
                
                document.save(filename)
                messagebox.showinfo("Success", f"Document exported successfully to:\n{filename}")
            except Exception as e:
                messagebox.showerror("Error", f"Failed to export to Word:\n{str(e)}")
    
    def export_to_pdf(self):
        """Export document to PDF format"""
        if not PDF_AVAILABLE:
            messagebox.showerror("Error", "reportlab not installed.\nInstall with: pip install reportlab")
            return
        
        doc_text = self.output_text.get("1.0", "end-1c")
        if not doc_text or doc_text.startswith("Select a document"):
            messagebox.showwarning("Warning", "Please generate a document first.")
            return
        
        filename = filedialog.asksaveasfilename(
            title="Save as PDF",
            defaultextension=".pdf",
            filetypes=[("PDF Document", "*.pdf"), ("All files", "*.*")]
        )
        
        if filename:
            try:
                doc = SimpleDocTemplate(filename, pagesize=A4)
                story = []
                styles = getSampleStyleSheet()
                
                # Create custom style
                normal_style = ParagraphStyle(
                    'CustomNormal',
                    parent=styles['Normal'],
                    fontName='Courier',
                    fontSize=10,
                    leading=14,
                    alignment=TA_LEFT
                )
                
                # Add content
                for line in doc_text.split('\n'):
                    if line.strip():
                        para = Paragraph(line.replace('<', '&lt;').replace('>', '&gt;'), normal_style)
                        story.append(para)
                    else:
                        story.append(Spacer(1, 0.2*inch))
                
                doc.build(story)
                messagebox.showinfo("Success", f"Document exported successfully to:\n{filename}")
            except Exception as e:
                messagebox.showerror("Error", f"Failed to export to PDF:\n{str(e)}")
    
    def email_document(self):
        """Send document via email"""
        if not EMAIL_AVAILABLE:
            messagebox.showerror("Error", "Email functionality not available.")
            return
        
        doc_text = self.output_text.get("1.0", "end-1c")
        if not doc_text or doc_text.startswith("Select a document"):
            messagebox.showwarning("Warning", "Please generate a document first.")
            return
        
        # Create email dialog
        email_dialog = tk.Toplevel(self.root)
        email_dialog.title("Send Email")
        email_dialog.geometry("500x600")
        email_dialog.configure(bg="white")
        
        # Email form
        tk.Label(email_dialog, text="Send Document via Email", font=("Arial", 14, "bold"), bg="white").pack(pady=10)
        
        # From email
        tk.Label(email_dialog, text="From (Your Email):", font=("Arial", 10), bg="white", anchor="w").pack(fill="x", padx=20, pady=(10, 5))
        from_email = tk.Entry(email_dialog, font=("Arial", 10))
        from_email.pack(fill="x", padx=20)
        
        # Password
        tk.Label(email_dialog, text="App Password:", font=("Arial", 10), bg="white", anchor="w").pack(fill="x", padx=20, pady=(10, 5))
        password = tk.Entry(email_dialog, font=("Arial", 10), show="*")
        password.pack(fill="x", padx=20)
        
        # To email
        tk.Label(email_dialog, text="To (Recipient Email):", font=("Arial", 10), bg="white", anchor="w").pack(fill="x", padx=20, pady=(10, 5))
        to_email = tk.Entry(email_dialog, font=("Arial", 10))
        to_email.pack(fill="x", padx=20)
        
        # Subject
        tk.Label(email_dialog, text="Subject:", font=("Arial", 10), bg="white", anchor="w").pack(fill="x", padx=20, pady=(10, 5))
        subject = tk.Entry(email_dialog, font=("Arial", 10))
        subject.insert(0, f"Document: {self.current_doc_type}")
        subject.pack(fill="x", padx=20)
        
        # Message
        tk.Label(email_dialog, text="Message:", font=("Arial", 10), bg="white", anchor="w").pack(fill="x", padx=20, pady=(10, 5))
        message = tk.Text(email_dialog, font=("Arial", 10), height=8)
        message.insert("1.0", "Please find the attached document.")
        message.pack(fill="both", expand=True, padx=20)
        
        # Attachment option
        attach_var = tk.BooleanVar(value=False)
        tk.Checkbutton(
            email_dialog,
            text="Attach as PDF (requires reportlab)",
            variable=attach_var,
            font=("Arial", 10),
            bg="white"
        ).pack(pady=10)
        
        # Info label
        info_label = tk.Label(
            email_dialog,
            text="Note: For Gmail, use App Password from Google Account settings",
            font=("Arial", 8),
            fg="#6B7280",
            bg="white",
            wraplength=450
        )
        info_label.pack(pady=5)
        
        def send_email():
            try:
                from_addr = from_email.get().strip()
                to_addr = to_email.get().strip()
                pwd = password.get().strip()
                subj = subject.get().strip()
                msg_text = message.get("1.0", "end-1c").strip()
                
                if not all([from_addr, to_addr, pwd, subj]):
                    messagebox.showwarning("Warning", "Please fill all required fields.")
                    return
                
                # Create message
                msg = MIMEMultipart()
                msg['From'] = from_addr
                msg['To'] = to_addr
                msg['Subject'] = subj
                
                # Add body
                body = msg_text + "\n\n" + "-"*50 + "\n\n" + doc_text
                msg.attach(MIMEText(body, 'plain'))
                
                # Attach PDF if requested
                if attach_var.get() and PDF_AVAILABLE:
                    import tempfile
                    with tempfile.NamedTemporaryFile(delete=False, suffix='.pdf') as tmp:
                        tmp_filename = tmp.name
                    
                    # Generate PDF
                    doc = SimpleDocTemplate(tmp_filename, pagesize=A4)
                    story = []
                    styles = getSampleStyleSheet()
                    normal_style = ParagraphStyle(
                        'CustomNormal',
                        parent=styles['Normal'],
                        fontName='Courier',
                        fontSize=10,
                        leading=14
                    )
                    
                    for line in doc_text.split('\n'):
                        if line.strip():
                            para = Paragraph(line.replace('<', '&lt;').replace('>', '&gt;'), normal_style)
                            story.append(para)
                        else:
                            story.append(Spacer(1, 0.2*inch))
                    
                    doc.build(story)
                    
                    # Attach file
                    with open(tmp_filename, 'rb') as f:
                        part = MIMEBase('application', 'octet-stream')
                        part.set_payload(f.read())
                        encoders.encode_base64(part)
                        part.add_header('Content-Disposition', f'attachment; filename=document.pdf')
                        msg.attach(part)
                    
                    os.unlink(tmp_filename)
                
                # Send email using Gmail SMTP
                server = smtplib.SMTP('smtp.gmail.com', 587)
                server.starttls()
                server.login(from_addr, pwd)
                server.send_message(msg)
                server.quit()
                
                messagebox.showinfo("Success", "Email sent successfully!")
                email_dialog.destroy()
                
            except Exception as e:
                messagebox.showerror("Error", f"Failed to send email:\n{str(e)}")
        
        # Send button
        send_btn = tk.Button(
            email_dialog,
            text="Send Email",
            font=("Arial", 12, "bold"),
            bg="#4F46E5",
            fg="white",
            relief="flat",
            padx=20,
            pady=10,
            command=send_email,
            cursor="hand2"
        )
        send_btn.pack(pady=10)
    
    def format_number(self, num_str):
        try:
            num = float(num_str)
            return f"{num:,.2f}"
        except:
            return "0.00"
    
    def number_to_words(self, num_str):
        try:
            num = int(float(num_str))
        except:
            return "Zero"
        
        if num == 0:
            return "Zero"
        
        ones = ['', 'One', 'Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine']
        tens = ['', '', 'Twenty', 'Thirty', 'Forty', 'Fifty', 'Sixty', 'Seventy', 'Eighty', 'Ninety']
        teens = ['Ten', 'Eleven', 'Twelve', 'Thirteen', 'Fourteen', 'Fifteen', 
                 'Sixteen', 'Seventeen', 'Eighteen', 'Nineteen']
        
        if num < 10:
            return ones[num]
        elif num < 20:
            return teens[num - 10]
        elif num < 100:
            return tens[num // 10] + ('' if num % 10 == 0 else ' ' + ones[num % 10])
        elif num < 1000:
            return ones[num // 100] + ' Hundred' + ('' if num % 100 == 0 else ' and ' + self.number_to_words(str(num % 100)))
        elif num < 100000:
            return self.number_to_words(str(num // 1000)) + ' Thousand' + ('' if num % 1000 == 0 else ' ' + self.number_to_words(str(num % 1000)))
        elif num < 10000000:
            return self.number_to_words(str(num // 100000)) + ' Lakh' + ('' if num % 100000 == 0 else ' ' + self.number_to_words(str(num % 100000)))
        else:
            return self.number_to_words(str(num // 10000000)) + ' Crore' + ('' if num % 10000000 == 0 else ' ' + self.number_to_words(str(num % 10000000)))
    
    def copy_to_clipboard(self):
        doc_text = self.output_text.get("1.0", "end-1c")
        self.root.clipboard_clear()
        self.root.clipboard_append(doc_text)
        messagebox.showinfo("Success", "Document copied to clipboard!")


def main():
    root = tk.Tk()
    app = DocumentAutomationSystem(root)
    root.mainloop()


if __name__ == "__main__":
    main()